<a href="https://colab.research.google.com/github/MarkPopovich/financial-ml/blob/experiments/numerai/Numerai_0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from IPython import display
!pip install gcsfs numerapi
display.clear_output()

In [20]:
from google.colab import drive
from google.cloud import storage
import pandas as pd
import time
import numerapi
import numpy as np
import os
import tensorflow as tf

In [21]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
tpu_service_account = 'grpc://' + os.environ['COLAB_TPU_ADDR'].split(':')[0]
tpu_service_account

# #!gsutil acl ch -u tpu-service-account:READER gs://bucket-name

INFO:tensorflow:Initializing the TPU system: grpc://10.57.227.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.57.227.170:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


'grpc://10.57.227.170'

In [22]:
bucket_path = 'gs://fin-aml/data/numerai/'
filename = 'numerai_train.tfrecord'
bucket = 'gs://fin-aml'

In [23]:
drive.mount('/content/drive')

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/drive/My Drive/fin-aml/tpu-training-289520-f7727af0669b.json"

storage_client = storage.Client()
display.clear_output()

In [24]:
bucket_path = 'gs://fin-aml/data/numerai/'
filename = 'numerai_train.tfrecord'
bucket_path + filename

'gs://fin-aml/data/numerai/numerai_train.tfrecord'

In [112]:
def get_dataset(filename):
    dataset = tf.data.TFRecordDataset(filename)

    dataset_key = {
                # 'id': tf.io.FixedLenFeature([], tf.string, default_value=''),
                # 'era': tf.io.FixedLenFeature([], tf.string, default_value=''),
                # 'kind': tf.io.FixedLenFeature([], tf.string, default_value=''),
                'X': tf.io.FixedLenFeature([310,], tf.float32),
                'target': tf.io.FixedLenFeature([1], tf.float32)
                }

    def _parse_xy_function(example_proto):
        proto = tf.io.parse_single_example(example_proto, dataset_key)
        return proto['X'], proto['target']
        #tf.one_hot(tf.cast(proto['target']*4, dtype=tf.int32), 5)

    return dataset.map(_parse_xy_function)

In [71]:
filename = 'numerai_training.tfrecord'
dataset = get_dataset(bucket_path+filename)

In [72]:
for x, y in dataset.batch(8).take(1):
  print(x, y)

tf.Tensor(
[[0.   0.5  0.25 ... 1.   0.5  0.75]
 [0.   0.   0.   ... 0.   0.25 1.  ]
 [0.25 0.5  0.25 ... 0.   0.25 0.75]
 ...
 [0.5  0.5  0.25 ... 0.   0.25 0.  ]
 [0.5  0.25 0.25 ... 0.5  0.5  1.  ]
 [0.25 1.   1.   ... 0.75 0.5  0.25]], shape=(8, 310), dtype=float32) tf.Tensor(
[[0.75]
 [0.25]
 [0.  ]
 [0.  ]
 [0.75]
 [0.25]
 [0.  ]
 [0.  ]], shape=(8, 1), dtype=float32)


In [74]:
filename = 'numerai_tournament_1.tfrecord'
dataset = get_dataset(bucket_path+filename) 

for x, y in dataset.batch(8).take(1):
  print(x, y)

tf.Tensor(
[[0.25 0.75 0.5  ... 1.   0.   0.  ]
 [0.75 0.5  0.75 ... 0.5  0.5  0.5 ]
 [1.   0.   0.   ... 0.5  0.5  0.  ]
 ...
 [0.75 0.75 0.75 ... 0.   0.   0.  ]
 [0.5  1.   1.   ... 1.   0.   0.75]
 [0.   1.   0.75 ... 1.   0.5  1.  ]], shape=(8, 310), dtype=float32) tf.Tensor(
[[0.  ]
 [0.25]
 [1.  ]
 [0.75]
 [1.  ]
 [0.75]
 [0.25]
 [0.5 ]], shape=(8, 1), dtype=float32)


In [91]:
def create_model():
  return tf.keras.Sequential([
        # [tf.keras.layers.Input(shape=(310,)),
        # tf.keras.layers.experimental.RandomFourierFeatures(
        #                             output_dim=1028,
        #                             scale=10.,
        #                             kernel_initializer='laplacian',
        #                             trainable=True),
        tf.keras.layers.Reshape((310,1)),
        tf.keras.layers.LSTM(256), 
        tf.keras.layers.Reshape((256,1)),
        tf.keras.layers.LSTM(256), 
        tf.keras.layers.Dense(5, activation='softmax')]
  )

# def create_model():
#   return tf.keras.Sequential(
#         [tf.keras.layers.Dense(5, activation='softmax')]
#   )

In [31]:
use_tpu = True

if use_tpu == True:
  strategy = tf.distribute.TPUStrategy(resolver)
else:
  strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [32]:
training_data_shape = (501808, 314)
test_data_shape = (1638063, 314)

In [39]:
training_data_shape[0] / (128 * 1000)

3.920375

In [38]:
test_data_shape[0] / (128 * 1000)

12.7973671875

In [113]:
with strategy.scope():
    model = create_model()
    #opt = tf.keras.optimizers.Adam(learning_rate=.1)
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  experimental_steps_per_execution = 400,
                  metrics=['sparse_categorical_accuracy','sparse_categorical_crossentropy'])
  
batch_size = 128 * 8
steps_per_epoch = 400
validation_steps = 400

train_file = 'numerai_training.tfrecord'
test_file = 'numerai_tournament_1.tfrecord'

train_dataset = get_dataset(bucket_path+train_file).batch(batch_size).repeat()
test_dataset = get_dataset(bucket_path+test_file).batch(batch_size).repeat()

model.fit(train_dataset,
          epochs=50,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps)

Epoch 1/50
400/400 [==============================] - 58s 144ms/step - loss: nan - sparse_categorical_accuracy: 0.1993 - sparse_categorical_crossentropy: nan - val_loss: nan - val_sparse_categorical_accuracy: 0.0274 - val_sparse_categorical_crossentropy: nan
Epoch 2/50


KeyboardInterrupt: ignored

In [ ]:
model.fit(train_dataset,
          epochs=100,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps)

In [ ]:
def create_model():
  return tf.keras.Sequential([
                              tf.keras.layers.Reshape((310,1)),
                              tf.keras.layers.LSTM(310, activation='relu'),
                              tf.keras.layers.Dense(1)]
                              )


with strategy.scope():
    model = create_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=.1)
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.MeanSquaredLogarithmicError(),
                  experimental_steps_per_execution = 400,
                  metrics=['mean_squared_logarithmic_error','mean_squared_error'])
  
batch_size = 64 
steps_per_epoch = 3200
validation_steps = 3200

train_file = 'numerai_training.tfrecord'
test_file = 'numerai_tournament_1.tfrecord'

train_dataset = get_dataset(bucket_path+train_file).batch(batch_size, drop_remainder=True).repeat()
test_dataset = get_dataset(bucket_path+test_file).batch(batch_size, drop_remainder=True).repeat()

model.fit(train_dataset,
          epochs=150,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps)

Epoch 1/150


 800/3200 [======>.......................] - ETA: 38s - loss: nan - mean_squared_logarithmic_error: nan - mean_squared_error: nan

In [45]:
def create_model():
  model = tf.keras.Sequential([tf.keras.layers.Dense(5, activation='softmax')])
  optimizer = tf.keras.optimizers.Adam(learning_rate=100)
  model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['sparse_categorical_accuracy','sparse_categorical_crossentropy'])
  return model
    
training_model = create_model()

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

history = tpu_model.fit(train_dataset,
                        epochs=20,
                        batch_size=128 * 8,
                        validation_split=0.2)
tpu_model.save_weights('./tpu_model.h5', overwrite=True)
tpu_model.evaluate(test_dataset, batch_size=128 * 8)

AttributeError: ignored

In [108]:
feature_columns = []
for feature_name in range(310):
  feature_columns.append(tf.feature_column.numeric_column(str(feature_name),
                                           dtype=tf.float32))

In [107]:
def make_input_fn(filename):
  """Returns an `input_fn` for train and eval."""

  def input_fn(params):
    """An input_fn to parse 28x28 images from filename using tf.data."""
    batch_size = 128 * 8

    def get_dataset(filename):
        dataset = tf.data.TFRecordDataset(filename)

        dataset_key = {
                    # 'id': tf.io.FixedLenFeature([], tf.string, default_value=''),
                    # 'era': tf.io.FixedLenFeature([], tf.string, default_value=''),
                    # 'kind': tf.io.FixedLenFeature([], tf.string, default_value=''),
                    'X': tf.io.FixedLenFeature([310,], tf.float32),
                    'target': tf.io.FixedLenFeature([1], tf.float32)
                    }

        def _parse_xy_function(example_proto):
            proto = tf.io.parse_single_example(example_proto, dataset_key)
            return proto['X'], proto['target']*4
            #tf.one_hot(tf.cast(proto['target']*4, dtype=tf.int32), 5)

        return dataset.map(_parse_xy_function)

    dataset = get_dataset(filename)

    dataset = dataset.batch(batch_size).repeat()
    
    return dataset

  return input_fn

In [105]:
with strategy.scope():
    model = tf.estimator.BoostedTreesClassifier(feature_columns, n_batches_per_layer=3200, n_classes=5)
    #opt = tf.keras.optimizers.Adam(learning_rate=.1)
    # model.compile(optimizer='adam',
    #               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    #               experimental_steps_per_execution = 400,
    #               metrics=['sparse_categorical_accuracy','sparse_categorical_crossentropy'])
  
batch_size = 128 * 8
steps_per_epoch = 400
validation_steps = 400

train_file = 'numerai_training.tfrecord'
test_file = 'numerai_tournament_1.tfrecord'

# train_dataset = get_dataset(bucket_path+train_file).batch(batch_size).repeat()
# test_dataset = get_dataset(bucket_path+test_file).batch(batch_size).repeat()

model.train(make_input_fn(train_file))

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpci4eywmv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpci4eywmv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


AttributeError: ignored

In [ ]:
test = get_dataset(bucket_path + test_file).batch(batch_size).take(1)

In [ ]:
for x, y in test:
  print(x.shape)
  print(y.shape)

In [ ]:
predictions = model.predict(test)

In [ ]:
predictions.shape

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y, predictions)

In [ ]:
y_df = pd.DataFrame(y.numpy())

In [ ]:
def get_ids(filename):
    dataset = tf.data.TFRecordDataset(filename)

    dataset_key = {
                'id': tf.io.FixedLenFeature([], tf.string, default_value=''),
                # 'era': tf.io.FixedLenFeature([], tf.string, default_value=''),
                # 'kind': tf.io.FixedLenFeature([], tf.string, default_value=''),
                # 'X': tf.io.FixedLenFeature([310,], tf.float32),
                # 'target': tf.io.FixedLenFeature([], tf.float32)
                }

    def _parse_xy_function(example_proto):
        proto = tf.io.parse_single_example(example_proto, dataset_key)
        return proto['id']

    return dataset.map(_parse_xy_function)

In [ ]:
ids = get_ids(bucket_path + test_file).batch(1024).take(1)

ids

In [ ]:
for id in ids:
  print(id)

In [ ]:
df = pd.DataFrame([id.numpy(), y.numpy()[:,0], predictions[:, 0]]).T

In [ ]:
df[0] = df[0].str.decode("utf-8")

In [ ]:
df.rename({0:'id',1:'y_true',2:'y_pred'},axis=1, inplace=True)

In [ ]:
df.set_index('id', inplace=True)

In [ ]:
np.corrcoef(df['y_true'].astype('float'), df['y_pred'].astype('float'))

In [ ]:
df['y_pred'].astype('float').describe()

In [ ]:
df['y_pred'].dtype

In [ ]:
indices = np.array([0, 0.25, .5, .75, 1.0]) * 4
depth = 5
print(indices)
tf.one_hot(indices, depth)

In [ ]:
tf.one_hot(tf.cast(y*4, dtype=tf.int32), 5)

In [ ]:
y*4

In [ ]:
def model_fn(features, labels, mode, params):
  """A simple CNN."""
  del params  # unused

  logits = tf.layers.dense(inputs=features, units=5, activation=tf.nn.softmax)

  loss = tf.losses.sparse_softmax_cross_entropy(
      labels=labels, logits=logits)

  learning_rate = tf.train.exponential_decay(
      FLAGS.learning_rate, tf.train.get_global_step(), 100000, 0.96)

  optimizer = tpu_optimizer.CrossShardOptimizer(
      tf.train.GradientDescentOptimizer(learning_rate=learning_rate))

  train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
  return tpu_estimator.TPUEstimatorSpec(mode=mode, loss=loss, train_op=train_op)


def make_input_fn(filename):
  """Returns an `input_fn` for train and eval."""

  def input_fn(params):
    """An input_fn to parse 28x28 images from filename using tf.data."""
    batch_size = params["batch_size"]

    def get_dataset(filename):
        dataset = tf.data.TFRecordDataset(filename)

        dataset_key = {
                    # 'id': tf.io.FixedLenFeature([], tf.string, default_value=''),
                    # 'era': tf.io.FixedLenFeature([], tf.string, default_value=''),
                    # 'kind': tf.io.FixedLenFeature([], tf.string, default_value=''),
                    'X': tf.io.FixedLenFeature([310,], tf.float32),
                    'target': tf.io.FixedLenFeature([1], tf.float32)
                    }

        def _parse_xy_function(example_proto):
            proto = tf.io.parse_single_example(example_proto, dataset_key)
            return proto['X'], proto['target']*4
            #tf.one_hot(tf.cast(proto['target']*4, dtype=tf.int32), 5)

        return dataset.map(_parse_xy_function)

    dataset = get_dataset(filename)

    dataset = dataset.repeat()
    dataset = dataset.apply(
      tf.contrib.data.map_and_batch(
         parser, batch_size=batch_size,
         num_parallel_batches=8,
         drop_remainder=True))
    
    return dataset

  return input_fn




In [ ]:


tf.logging.set_verbosity(tf.logging.INFO)

run_config = tpu_config.RunConfig(
    master=FLAGS.master,
    model_dir=FLAGS.model_dir,
    session_config=tf.ConfigProto(
        allow_soft_placement=True, log_device_placement=True),
    tpu_config=tpu_config.TPUConfig(FLAGS.iterations))

estimator = tpu_estimator.TPUEstimator(
    model_fn=model_fn,
    use_tpu=FLAGS.use_tpu,
    train_batch_size=FLAGS.batch_size,
    eval_batch_size=FLAGS.batch_size,
    config=run_config)

estimator.train(input_fn=make_input_fn(FLAGS.train_file),
                max_steps=FLAGS.train_steps)